In [1]:
#Build Molecule
import numpy as np
from pyscf import gto, scf, lib, mcscf
import math
import time
import os
from tools import sign_control, util
from systems.niochain import NiOChain

def calc_gap(dist=2,nio_dist=1.6,fn="logs/nio.log"):
    #dist -- distance between NiO fragments
    #nio_dist -- Ni O distance
    geom = NiOChain(4,dist=dist,nio_dist=nio_dist,fn=fn)
    las = geom.make_las_state_average()
    
    #Build LAS states
    print("Initiating LAS Model Space...")
    start = time.perf_counter()
    las.lasci_()
    las.ci = sign_control.fix_sign(las)
    stop = time.perf_counter()
    print("Done. Time elapsed:",stop-start,"seconds")
    las.dump_spaces()
        
    #Run LASSI
    print("Running LASSI...")
    start = time.perf_counter()
    from mrh.my_pyscf import lassi
    lsi = lassi.LASSI(las)
    energies_lassi, civecs_lassi = lsi.kernel()
    civecs_lassi = lsi.get_sivec_vacuum_shuffle(state=0) #Sign fixing
    stop = time.perf_counter()
    print("Done. Time elapsed:",stop-start,"seconds")
    
    hartree_to_ev = 27.2114
    de = energies_lassi[1] - energies_lassi[0]
    gap = de * hartree_to_ev
    print("Gap:",np.round(gap,2),"eV")
    return gap

calc_gap(dist=2.5,nio_dist=2.1)

/home/king1305/Apps/pyscf/pyscf/dft/libxc.py:772: UserWarning: Since PySCF-2.3, B3LYP (and B3P86) are changed to the VWN-RPA variant, corresponding to the original definition by Stephens et al. (issue 1480) and the same as the B3LYP functional in Gaussian. To restore the VWN5 definition, you can put the setting "B3LYP_WITH_VWN5 = True" in pyscf_conf.py
  warnings.warn('Since PySCF-2.3, B3LYP (and B3P86) are changed to the VWN-RPA variant, '


overwrite output file: logs/nio.log
Initiating LAS Model Space...
Done. Time elapsed: 21.68956315901596 seconds
Running LASSI...
Done. Time elapsed: 0.41217574395705014 seconds
Gap: 3.81 eV


3.8108463395284278